In [ ]:
!pip install transformers --quiet

In [2]:
!id="18nemaqdNT-0ZtEWyD57a6tO3LXUcgtMj"; conf=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id='$id -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p'); wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$conf&id=$id" -O covid_fake_news.csv && rm -rf /tmp/cookies.txt

--2022-08-25 12:41:11--  https://docs.google.com/uc?export=download&confirm=&id=18nemaqdNT-0ZtEWyD57a6tO3LXUcgtMj
Resolving docs.google.com (docs.google.com)... 142.250.186.206, 2a00:1450:401b:80e::200e
Connecting to docs.google.com (docs.google.com)|142.250.186.206|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-00-34-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qovriuqek2b1t5a9u0d4c8i6pfiqjbm9/1661424000000/05536367513713019519/*/18nemaqdNT-0ZtEWyD57a6tO3LXUcgtMj?e=download&uuid=8af1858f-825c-4f7d-9a17-50ce4fdfb6a1 [following]
--2022-08-25 12:41:12--  https://doc-00-34-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/qovriuqek2b1t5a9u0d4c8i6pfiqjbm9/1661424000000/05536367513713019519/*/18nemaqdNT-0ZtEWyD57a6tO3LXUcgtMj?e=download&uuid=8af1858f-825c-4f7d-9a17-50ce4fdfb6a1
Resolving doc-00-34-docs.googleusercontent.com (doc-00-34-docs.googleusercontent.com)... 142.250.75.1, 2a00:1450:40

In [2]:
data["text"] = X
data["label"] = 1-y
data.to_csv("raw/covid_fake_news.csv",sep="\t")

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import RepeatedStratifiedKFold

data = pd.read_csv("../../raw/covid_fake_news.csv",sep="\t")
print("labels\n",data["label"].value_counts())
print("shape  \n",data.shape)

bootstrap_size = 0

if bootstrap_size != 0:
    bootstrap_factor = bootstrap_size / data.shape[0]
    bootstrap = np.random.uniform(size=data.shape[0]) < bootstrap_factor
    data = data.iloc[bootstrap]

X, y = data["title"].values, data["label"].values

rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=5, random_state=1410)
foldids = []
for fold_idx, (train, test) in tqdm(enumerate(rskf.split(X, y)), total=rskf.get_n_splits()):
    foldids.append((fold_idx,train,test))
    
print("shapes X",X.shape,"y", y.shape)

labels
 1    8511
0     461
Name: label, dtype: int64
shape  
 (8972, 4)


100%|█████████████████████████████████████████| 10/10 [00:00<00:00, 1512.01it/s]

shapes X (8972,) y (8972,)


In [4]:
data.columns

Index(['Unnamed: 0', 'title', 'label', 'text'], dtype='object')

In [10]:
import transformers
transformers.logging.set_verbosity_error()
from torch.utils.data import DataLoader 

from transformers import AutoTokenizer
from transformers import AutoModel
from transformers import TFDistilBertModel, DistilBertTokenizerFast

from sklearn.base import BaseEstimator, TransformerMixin
import torch
import numpy as np

from scipy.sparse import csr_matrix
from tqdm import tqdm

from sklearn.linear_model import LogisticRegression

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print("using device:", device)

if "disilbert_model" not in locals():
    disilbert_tokenizer =  AutoTokenizer.from_pretrained("distilbert-base-uncased")
    disilbert_model = AutoModel.from_pretrained("distilbert-base-uncased")
    handle = disilbert_model.to(device)


class BERTEmbeddings(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tokenizer =  disilbert_tokenizer
        self.model = disilbert_tokenizer
        self.max_length = 256
        self.model_name = disilbert_model

    def fit(self, X=None, y=None):
        pass
    
    def encode(self, txt):
        return self.tokenizer(txt, max_length=self.max_length, 
                              truncation=True, padding=True, return_tensors="pt")

    def transform(self, X=None):
        dataloader = DataLoader(X, batch_size=4, shuffle=False)
        allembeds = []
        for batch in tqdm(dataloader):
            batchenc = disilbert_tokenizer(batch, max_length=256, truncation=True, padding=True, return_tensors="pt")
            input_ids = batchenc['input_ids'].to(device)
            attention_mask = batchenc['attention_mask'].to(device)
            batchout = disilbert_model(input_ids, attention_mask=attention_mask, output_hidden_states=True)
            embeds = [vec[0].cpu().detach().numpy() for vec in batchout[1][-1]]
            allembeds.extend(embeds)
        return csr_matrix(allembeds)


class BertHead(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.head = LogisticRegression(class_weight='auto', max_iter=10000)

    def fit(self, X=None, y=None):
        self.head.fit(X, y)


    def transform(self, X=None):
        pass
    
    def predict(self, X=None):    
        return self.head.predict(X)
    
    def predict_proba(self, X=None):    
        return self.head.predict_proba(X)



using device: cuda


In [5]:
bert = BERTEmbeddings()
X_dstil_numpy = bert.transform(X).toarray()

100%|████████████████████████████████████████| 561/561 [00:04<00:00, 134.35it/s]


In [6]:
import pickle
# SAVE
with open('covid_fake_news_BERTEmbeddings.pickle', 'wb') as handle:
    pickle.dump(X_dstil_numpy, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
import pickle
# LOAD
with open('../../pickles/bertcls_embeddings/covid_fake_news_BERTEmbeddings.pickle', 'rb') as handle:
    X_dstil_numpy=pickle.load(handle)

In [11]:
from sklearn.metrics import log_loss,accuracy_score, balanced_accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from imblearn.metrics import geometric_mean_score
from IPython.display import clear_output

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def log_los_nonorm(x,y,average=False):
    return log_loss(x,y,normalize=True)

scores = {
    'Accuracy': {'func': accuracy_score},
    'Balanced Accuracy': {'func': balanced_accuracy_score},
    'F1': {'func': f1_score},
    'Precision': {'func': precision_score},
    'Recall': {'func': recall_score},
    'Log-likelihood': {'func': log_los_nonorm},
    #'G-mean': {'func': geometric_mean_score}
}

for score_name, score_dict in scores.items():
    scores[score_name]["list"] = []
    scores[score_name]["lab"] = []


for fold,j in enumerate(foldids):
    train = foldids[fold][1]
    test = foldids[fold][2]
    xin, yin = X_dstil_numpy[train], np.array(y[train])
    cls = BertHead()
    
    cls.fit(xin, yin)
    y_pred = cls.predict(X_dstil_numpy[test])
    y_pred_proba = cls.predict_proba(X_dstil_numpy[test])
    
    for score_name, score_dict in scores.items():
        if score_name == "Log-likelihood":
            scorval=score_dict['func'](y[test], y_pred_proba)
            score_dict['list'].append(scorval)
            print(score_name, scorval)
        elif score_name in ["F1","Precision","Recall"]:
            scorvaln = score_dict['func'](y[test], y_pred, average=None)
            score_dict['lab'].append(scorvaln)
            scorval = score_dict['func'](y[test], y_pred, average="weighted")
            score_dict['list'].append(scorval)
            print(score_name, scorval, scorvaln)  
        else:
            scorval=score_dict['func'](y[test], y_pred)
            score_dict['list'].append(scorval)
            print(score_name, scorval)
    print()

Accuracy 0.9719126170307624
Balanced Accuracy 0.8083442301405688
F1 0.9704276929382857 [0.69565217 0.98527693]
Precision 0.9698884467882206 [0.7826087 0.9800093]
Recall 0.9719126170307624 [0.62608696 0.9906015 ]
Log-likelihood 0.08265961043378738

Accuracy 0.9736959429335711
Balanced Accuracy 0.7773406382102034
F1 0.9708193476806144 [0.68617021 0.98627268]
Precision 0.972031002536266 [0.88965517 0.97650311]
Recall 0.9736959429335711 [0.55844156 0.99623972]
Log-likelihood 0.07848217226762058

Accuracy 0.9719126170307624
Balanced Accuracy 0.8104006619810395
F1 0.9704994475326 [0.69711538 0.98527349]
Precision 0.9699444589190097 [0.77956989 0.98023256]
Recall 0.9719126170307624 [0.63043478 0.99036654]
Log-likelihood 0.07957285118617873

Accuracy 0.9728042799821668
Balanced Accuracy 0.7911995564169477
F1 0.9705652349131253 [0.69035533 0.98577757]
Precision 0.9706265111365114 [0.83435583 0.97802452]
Recall 0.9728042799821668 [0.58874459 0.99365452]
Log-likelihood 0.07854661879993707

Accura

In [24]:
numlabels = scores["F1"]["lab"][0].shape[0]
scores["F1"]["lab"][0].shape[0] 
head = "| %-20s | %-12s |" +  numlabels * " %-10s |" 
headv = ["Score", "Average"]
headv.extend(["Kat_"+str(i+1) for i in range(numlabels)])
row=head % tuple(headv)
#print("+"*len(row))
print(row)
#print("+"*len(row))

for score_name, score_dict in sorted(scores.items()) :
    if score_name == "Log-likelihood":
        headv = [score_name, np.mean(score_dict['list'])*1, np.std(score_dict['list'])*1]
    else:
        headv = [score_name, np.mean(score_dict['list'])*100, np.std(score_dict['list'])*100]
    
    for i in range(numlabels):
        if score_name == "Log-likelihood":
            head = "| %-20s | %3.2f ± %3.3f |" + numlabels * " %-10s |" 
            headv.append("-")
        elif score_name in ["F1","Precision","Recall"]:
            head = "| %-20s | %4.1f ± %3.3f |" + numlabels* " %4.1f ± %3.1f |"
            vals = [v[i] for v in scores[score_name]["lab"]]
            headv.append(np.mean(vals)*100)
            headv.append(np.std(vals)*100)
        else:
            head = "| %-20s | %4.1f ± %3.3f |" + numlabels * " %-10s |" 
            headv.append("-")
    print(head % tuple(headv))
    
#print("+"*len(row))

| Score                | Average      | Kat_1      | Kat_2      |
| Accuracy             | 97.3 ± 0.106 | -          | -          |
| Balanced Accuracy    | 79.4 ± 1.407 | -          | -          |
| F1                   | 97.0 ± 0.119 | 69.0 ± 1.5 | 98.6 ± 0.1 |
| Log-likelihood       | 0.08 ± 0.002 | -          | -          |
| Precision            | 97.0 ± 0.124 | 82.3 ± 3.2 | 97.8 ± 0.2 |
| Recall               | 97.3 ± 0.106 | 59.6 ± 2.9 | 99.3 ± 0.2 |
